In [1]:
import numpy as np
import pandas as pd
import subprocess
import argparse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from bs4 import BeautifulSoup
import pickle

import re


C:\Users\romai\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 1. Career Builder jobs processing

In [2]:
path =  "../../data/"
dataset = "cb12/"

raw_path = path + dataset + "raw/" 
interim_path = path + dataset + "interim/"
processed_path = path + dataset + "processed/"

In [3]:
# Read data
jobs = pd.read_csv(raw_path + "jobs.tsv", header=0, sep='\t', on_bad_lines='warn') #, error_bad_lines=False)
jobs = jobs.rename(columns={"JobID": "item_id", "State": "state", "Country": "country", "City": "city", "Zip5": "zip5"})
#jobs = jobs.set_index("item_id")

C:\Users\romai\AppData\Local\Temp\ipykernel_21528\3950337761.py:2: ParserWarning: Skipping line 122433: expected 11 fields, saw 12

  jobs = pd.read_csv(raw_path + "jobs.tsv", header=0, sep='\t', on_bad_lines='warn') #, error_bad_lines=False)
C:\Users\romai\AppData\Local\Temp\ipykernel_21528\3950337761.py:2: ParserWarning: Skipping line 602576: expected 11 fields, saw 12

  jobs = pd.read_csv(raw_path + "jobs.tsv", header=0, sep='\t', on_bad_lines='warn') #, error_bad_lines=False)
C:\Users\romai\AppData\Local\Temp\ipykernel_21528\3950337761.py:2: ParserWarning: Skipping line 990950: expected 11 fields, saw 12

  jobs = pd.read_csv(raw_path + "jobs.tsv", header=0, sep='\t', on_bad_lines='warn') #, error_bad_lines=False)
C:\Users\romai\AppData\Local\Temp\ipykernel_21528\3950337761.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs = pd.read_csv(raw_path + "jobs.tsv", header=0, sep='\t', on_bad_lines='warn') #, error_bad_lines=

In [4]:
jobs.head()

,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59


In [5]:
for i in [609681, 1010392]:
    display(jobs[jobs["item_id"]==i])

,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate


,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate
966698,1010392,6,Administrative Assistant,Administrative Assistant<BR>\r<BR>\rA very suc...,Bachelors degree required<BR>\rMicrosoft Offic...,Atlanta,GA,US,NaN,2012-05-29 10:56:40.517,2012-06-28 23:59:00


In [6]:
for i in ['609656', '154928', '609681', '180144', '481586', '481541']:
    display(jobs[jobs["item_id"]==i])

,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate


,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate


,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate


,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate


,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate


,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate


In [7]:
jobs["item_id"].dtype

dtype('int64')

In [8]:
import gc
gc.collect()

10

In [9]:
for i in [1098334, 1096879]:
    display(jobs[jobs["item_id"]==i])

,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate
858957,1098334,5,Customer Service Representative,"<P>AmeriCash Loans, LLC is a fast-growing fina...","<UL>\r <LI>Recent banking, sales, consumer ...",Chicago,IL,US,60628.0,2012-05-16 10:38:19.9,2012-06-15 23:59:00


,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate
280187,1096879,1,Desktop Support,<p><span>First Level Support </span></p>\r<p>&...,<p><span>C.01 Support Center Scope of Servi...,Morris,IL,US,60450,2012-03-19 10:23:59.117,2012-04-18 23:59:59


In [10]:
interactions = pd.read_csv("../../data/cb12/interim/merged_valid_interactions.csv", header=0, sep='\t') #interactions.csv", header=0, sep='\t')

interactions.head()

,Unnamed: 0,user_id,created_at,item_id,interaction_type,session_id
0,0,9,1337034642,809208,0,3
1,1,9,1337035117,136489,0,3
2,2,9,1337035689,617374,0,3
3,3,14,1337092157,206046,0,7
4,4,14,1337092390,787741,0,7


In [11]:
for i in [609681, 1010392]:
    display(interactions[interactions["item_id"]==i])

,Unnamed: 0,user_id,created_at,item_id,interaction_type,session_id


,Unnamed: 0,user_id,created_at,item_id,interaction_type,session_id


In [12]:
for i in [609656, 154928, 609681, 180144, 481586, 481541]:
    display(interactions[interactions["item_id"]==i])

,Unnamed: 0,user_id,created_at,item_id,interaction_type,session_id


,Unnamed: 0,user_id,created_at,item_id,interaction_type,session_id


,Unnamed: 0,user_id,created_at,item_id,interaction_type,session_id


,Unnamed: 0,user_id,created_at,item_id,interaction_type,session_id


,Unnamed: 0,user_id,created_at,item_id,interaction_type,session_id


,Unnamed: 0,user_id,created_at,item_id,interaction_type,session_id


In [13]:
# Get unique item_ids from interactions
unique_item_ids = interactions['item_id'].unique()
print(len(unique_item_ids))

print(len(jobs))
# Filter items where 'item_id' is in unique_item_ids
jobs = jobs[jobs["item_id"].isin(unique_item_ids)]
print(len(jobs))

34880
1091923
34880


In [14]:
for i in [609681, 1010392]:
    display(jobs[jobs["item_id"]==i])

,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate


,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate


In [15]:
jobs["Requirements"].fillna("", inplace=True)
jobs["Description"].fillna("", inplace=True)
jobs["Title"].fillna("", inplace=True)

#jobs['Requirements'] = [BeautifulSoup(text).get_text() for text in jobs['Requirements']]

jobs['Requirements'] = jobs['Requirements'].map(lambda x:re.sub('<[^<]+?>', '', x)).map(lambda x:re.sub('\\\\r', '', x)).map(lambda x:re.sub('\\\\n', '', x)).map(lambda x:re.sub('&nbsp;', ' ', x)).map(lambda x:re.sub('[—]+', ' ', x)).map(lambda x:re.sub('/', ' ', x))
jobs['Description'] = jobs['Description'].map(lambda x:re.sub('<[^<]+?>', '', x)).map(lambda x:re.sub('\\\\r', '', x)).map(lambda x:re.sub('\\\\n', '', x)).map(lambda x:re.sub('&nbsp;', ' ', x)).map(lambda x:re.sub('[—]+', ' ', x)).map(lambda x:re.sub('/', ' ', x))
jobs['Title'] = jobs['Title'].map(lambda x:re.sub('/', ' ', x))

jobs['Requirements'] = jobs['Requirements'].str.lower()
jobs['Description'] = jobs['Description'].str.lower()
jobs['Title'] = jobs['Title'].str.lower()

jobs.head()

C:\Users\romai\AppData\Local\Temp\ipykernel_21528\1367039093.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  jobs["Requirements"].fillna("", inplace=True)
C:\Users\romai\AppData\Local\Temp\ipykernel_21528\1367039093.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate
17,22,1,gallery sales positions,gallery sales positions ...,please refer to the job description to view th...,Orlando,FL,US,32819,2012-03-09 16:51:56.403,2012-04-08 23:59:59
51,60,1,driver,"roofing mat'l supplier needs drivers w cdl, 3 ...",please refer to the job description to view th...,Orlando,FL,US,32803,2012-03-22 01:26:00.61,2012-04-21 23:59:59
59,81,1,security guards,"jenkins security consultants ""we take s...",please refer to the job description to view th...,Orlando,FL,US,32801,2012-03-29 16:06:49.687,2012-04-28 23:59:59
84,244,1,practice manager,practice manager ba200906 ...,please refer to the job description to view th...,Baltimore,MD,US,21227,2012-03-14 07:30:32.817,2012-04-13 23:59:59
113,370,1,domestic violence,domestic violence and traumatic brain injury p...,please refer to the job description to view th...,Tallahassee,FL,US,32301,2012-03-11 01:48:40.597,2012-04-10 23:59:59


In [16]:
#jobs.to_csv(interim_path + "jobs_cleaned.tsv", sep='\t')
len(jobs.Requirements.unique())

26773

In [17]:
print("Unique cities: " + str(len(jobs.city.unique())))
#print(jobs['city'].value_counts(normalize=True) * 100)
print("Unique states: " + str(len(jobs.state.unique())))
#print(jobs['state'].value_counts(normalize=True) * 100)
print("Unique zip codes: " + str(len(jobs.zip5.unique())))
print("Unique countries: " + str(len(jobs.country.unique())))
#print(jobs['country'].value_counts(normalize=True) * 100)

Unique cities: 2746
Unique states: 51
Unique zip codes: 8070
Unique countries: 1


In [18]:
jobs.isnull().sum()

item_id             0
WindowID            0
Title               0
Description         0
Requirements        0
city                0
state               0
country             0
zip5            10660
StartDate           0
EndDate             0
dtype: int64

In [19]:
from sentence_transformers import SentenceTransformer

texts_title = jobs['Title'].tolist()
texts_desc = jobs['Description'].tolist()
texts_req  = jobs['Requirements'].tolist()

# Load pretrained Sentence-BERT model
sbert = SentenceTransformer('all-MiniLM-L6-v2')  # 384-dim output

In [20]:
texts_title = [str(t) if t else "" for t in texts_title]
texts_desc = [str(t) if t else "" for t in texts_desc]
texts_req  = [str(t) if t else "" for t in texts_req]


In [21]:
# Encodage
title_emb = sbert.encode(texts_title, show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/1090 [00:00<?, ?it/s]

In [22]:
np.isnan(title_emb).sum()

0

In [23]:
desc_emb  = sbert.encode(texts_desc, show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/1090 [00:00<?, ?it/s]

In [24]:
req_emb   = sbert.encode(texts_req, show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/1090 [00:00<?, ?it/s]

In [25]:
jobs = jobs.reset_index(drop=True)
for i in [1098334, 1096879]:
    display(jobs[jobs["item_id"]==i])

,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate
33666,1098334,5,customer service representative,"americash loans, llc is a fast-growing financi...","recent banking, sales, consumer finance or...",Chicago,IL,US,60628.0,2012-05-16 10:38:19.9,2012-06-15 23:59:00


,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,EndDate
9215,1096879,1,desktop support,first level support first level support will ...,c.01 support center scope of servicefirst l...,Morris,IL,US,60450,2012-03-19 10:23:59.117,2012-04-18 23:59:59


In [26]:
# Création des DataFrames avec colonnes nommées
title_df = pd.DataFrame(title_emb, columns=[f"title_emb_{i}" for i in range(title_emb.shape[1])])
desc_df  = pd.DataFrame(desc_emb,  columns=[f"desc_emb_{i}" for i in range(desc_emb.shape[1])])
req_df   = pd.DataFrame(req_emb,   columns=[f"req_emb_{i}" for i in range(req_emb.shape[1])])

# Fusion dans le DataFrame jobs
jobs = jobs.reset_index(drop=True)
jobs = pd.concat([jobs, title_df, desc_df, req_df], axis=1)

jobs.head()

,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,...,req_emb_374,req_emb_375,req_emb_376,req_emb_377,req_emb_378,req_emb_379,req_emb_380,req_emb_381,req_emb_382,req_emb_383
0,22,1,gallery sales positions,gallery sales positions ...,please refer to the job description to view th...,Orlando,FL,US,32819,2012-03-09 16:51:56.403,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
1,60,1,driver,"roofing mat'l supplier needs drivers w cdl, 3 ...",please refer to the job description to view th...,Orlando,FL,US,32803,2012-03-22 01:26:00.61,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
2,81,1,security guards,"jenkins security consultants ""we take s...",please refer to the job description to view th...,Orlando,FL,US,32801,2012-03-29 16:06:49.687,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
3,244,1,practice manager,practice manager ba200906 ...,please refer to the job description to view th...,Baltimore,MD,US,21227,2012-03-14 07:30:32.817,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
4,370,1,domestic violence,domestic violence and traumatic brain injury p...,please refer to the job description to view th...,Tallahassee,FL,US,32301,2012-03-11 01:48:40.597,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348


In [27]:
for i in [1098334, 1096879]:
    display(jobs[jobs["item_id"]==i])

,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,...,req_emb_374,req_emb_375,req_emb_376,req_emb_377,req_emb_378,req_emb_379,req_emb_380,req_emb_381,req_emb_382,req_emb_383
33666,1098334,5,customer service representative,"americash loans, llc is a fast-growing financi...","recent banking, sales, consumer finance or...",Chicago,IL,US,60628.0,2012-05-16 10:38:19.9,...,0.11446,-0.029539,0.009867,0.000001,0.044585,-0.006104,0.023191,-0.024687,-0.033928,-0.028893


,item_id,WindowID,Title,Description,Requirements,city,state,country,zip5,StartDate,...,req_emb_374,req_emb_375,req_emb_376,req_emb_377,req_emb_378,req_emb_379,req_emb_380,req_emb_381,req_emb_382,req_emb_383
9215,1096879,1,desktop support,first level support first level support will ...,c.01 support center scope of servicefirst l...,Morris,IL,US,60450,2012-03-19 10:23:59.117,...,-0.023333,0.040784,0.060435,-0.019933,0.057278,0.092444,-0.008459,0.008499,-0.006461,0.012677


In [28]:
assert len(jobs) == len(title_df)
assert not jobs[[f"title_emb_{i}" for i in range(10)]].isna().any().any()


In [29]:
#jobs.to_csv(interim_path + "nlp_items.csv", sep='\t')

In [30]:
jobs = jobs.drop(columns=["WindowID", "Title", "Description", "Requirements", "StartDate", "EndDate"])

jobs.to_csv(interim_path + "nlp_items.csv", sep='\t')
len(jobs)

34880

In [31]:
jobs.head()

,item_id,city,state,country,zip5,title_emb_0,title_emb_1,title_emb_2,title_emb_3,title_emb_4,...,req_emb_374,req_emb_375,req_emb_376,req_emb_377,req_emb_378,req_emb_379,req_emb_380,req_emb_381,req_emb_382,req_emb_383
0,22,Orlando,FL,US,32819,0.031234,-0.058801,-0.023616,-0.037820,-0.058765,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
1,60,Orlando,FL,US,32803,-0.022986,0.054471,-0.016137,0.042382,-0.015666,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
2,81,Orlando,FL,US,32801,-0.038087,0.080151,-0.052039,0.015201,0.042734,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
3,244,Baltimore,MD,US,21227,-0.012535,0.010516,-0.009069,0.006332,-0.070244,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
4,370,Tallahassee,FL,US,32301,-0.023253,0.037175,-0.022867,0.073320,-0.010799,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348


In [32]:
for c in jobs.columns:
    if jobs[c].isnull().sum()>0:
        print(c)

zip5


In [33]:
import gc
gc.collect()

0

In [34]:
jobs = pd.read_csv(interim_path + "nlp_items.csv", header=0, sep='\t')
print(len(jobs))
jobs.head()

34880


,Unnamed: 0,item_id,city,state,country,zip5,title_emb_0,title_emb_1,title_emb_2,title_emb_3,...,req_emb_374,req_emb_375,req_emb_376,req_emb_377,req_emb_378,req_emb_379,req_emb_380,req_emb_381,req_emb_382,req_emb_383
0,0,22,Orlando,FL,US,32819.0,0.031234,-0.058801,-0.023616,-0.037820,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
1,1,60,Orlando,FL,US,32803.0,-0.022986,0.054471,-0.016137,0.042382,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
2,2,81,Orlando,FL,US,32801.0,-0.038087,0.080151,-0.052039,0.015201,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
3,3,244,Baltimore,MD,US,21227.0,-0.012535,0.010516,-0.009069,0.006332,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348
4,4,370,Tallahassee,FL,US,32301.0,-0.023253,0.037175,-0.022867,0.073320,...,-0.032298,0.05112,0.013713,-0.029943,-0.031015,0.029654,0.019965,0.076117,0.038724,-0.024348


In [35]:
train_valid = pd.read_csv(processed_path + "train_valid_14d.csv", header=0, sep='\t')
print(len(train_valid))
train_valid.head()

45392


,Unnamed: 0,user_id,created_at,item_id,interaction_type,session_id
0,780806,9,1337034642,809208,0,3
1,780807,9,1337035117,136489,0,3
2,780808,9,1337035689,617374,0,3
3,780812,14,1337092157,206046,0,7
4,780813,14,1337092390,787741,0,7


In [36]:
count=0

for i in train_valid["item_id"].unique():
    if i not in jobs["item_id"].unique():
        print(i)
        count+=1
if count==0:
    print("all items=job IDs from test are in train")
print("END")


all items=job IDs from test are in train
END


In [37]:
del train_valid

In [38]:
test_valid = pd.read_csv(processed_path + "test_valid_14d.csv", header=0, sep='\t')
print(len(test_valid))
test_valid.head()

2031


,Unnamed: 0,user_id,created_at,item_id,interaction_type,session_id
0,1014894,1902,1338901722,466171,0,592
1,1014891,1902,1338901729,811982,0,592
2,1014895,1902,1338901741,1013337,0,592
3,1014896,1902,1338901748,565013,0,592
4,1014893,1902,1338901772,854862,0,592


In [39]:
count=0

for i in test_valid["item_id"].unique():
    if i not in jobs["item_id"].unique():
        print(i)
        count+=1
if count==0:
    print("all items=job IDs from test are in train")
print("END")


all items=job IDs from test are in train
END


In [40]:
count=0
test_list = [926802, 44221]
for i in test_list:
    if i not in jobs["item_id"].unique():
        print(i)
        count+=1
if count==0:
    print("all items=job IDs from test are in jobs")
print("END")


all items=job IDs from test are in jobs
END
